# Inverse static (captive test)

In [ ]:
# %load imports.py
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
np.set_printoptions(linewidth=150)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl

from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.parameters import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models import regression
from vessel_manoeuvring_models.visualization.regression import show_pred
from vessel_manoeuvring_models.visualization.plot import track_plot

## Load models:
# (Uncomment these for faster loading):
import vessel_manoeuvring_models.models.vmm_martin  as vmm 
from vessel_manoeuvring_models.models.vmm import ModelSimulator
from example_ship2 import ship_parameters, df_parameters

from vessel_manoeuvring_models.vct_scaling import scale_force_to_model_scale, scale_moment_to_model_scale

if os.name == 'nt':
    plt.style.use('../docs/book/book.mplstyle')  # Windows
    
from vessel_manoeuvring_models.visualization.plot import track_plot, plot, captive_plot

In [ ]:
from vessel_manoeuvring_models.models.regression import ForceRegression

In [ ]:
df_VCT = pd.read_csv('../data/external/vct.csv', index_col=0)
df_VCT = df_VCT.groupby('model_name').get_group('V2_5_MDL_modelScale')
df_VCT.head()

In [ ]:
data_scaled = df_VCT.copy()
ship_data = ship_parameters
ship_data['scale_factor'] = scale_factor = 41.2
keys_forces = ["fx", "fy", "fz"]
scale_factor = ship_data["scale_factor"]
forces = data_scaled[keys_forces]
data_scaled[keys_forces] = scale_force_to_model_scale(
    forces=forces, scale_factor=scale_factor, **data_scaled
)


keys_moments = ["mx", "my", "mz"]
data_scaled[keys_moments] = scale_moment_to_model_scale(
    data_scaled[keys_moments], scale_factor=scale_factor, **data_scaled
)

In [ ]:
df_VCT[['lpp','S','V','fy','mz']].abs().max()

In [ ]:
data_scaled[['lpp','S','V','fy']].abs().max()

In [ ]:
(df_VCT['fy']/scale_factor**3).mean()

In [ ]:
(df_VCT['mz']/scale_factor**4).mean()

In [ ]:
interesting = ['u','v','r','delta','thrust','fx','fy','mz']
data = df_VCT[interesting].copy()

In [ ]:
added_masses = df_parameters.groupby(by='state').get_group('dot')['prime'].dropna().to_dict()

In [ ]:
regression = ForceRegression(vmm=vmm, data=data, added_masses=added_masses, ship_parameters=ship_parameters, exclude_parameters={'Xthrust':1})

In [ ]:
regression.show()

In [ ]:
regression.parameters

In [ ]:
data.head()

In [ ]:
keys = set(data.columns) - set(['fx','fy','mz'])
    

In [ ]:
y_vars=['fx','fy','mz']
x_vars = list(set(data.columns) - set(y_vars))

data_plot = data.copy()
sns.pairplot(data_plot, x_vars=x_vars, y_vars=y_vars, height=2.5)

In [ ]:
df_VCT['item'] = 'VCT'
captive_plot(df_captive=df_VCT)